# F1‑API – Notebook de Testes **completo**

Este notebook cobre **todos os endpoints** da aplicação, com operações de **create, read, update, delete** e validações para cada entidade.

**Pré‐requisito:** servidor executando em `http://localhost:8000`.

---

In [1]:
import requests, pprint, json, random, datetime
BASE='http://localhost:8000/api'
pp = pprint.PrettyPrinter(compact=True).pprint

def show(title, resp):
    print(f'### {title} | status', resp.status_code)
    try:
        pp(resp.json())
    except Exception:
        print(resp.text)


## 1. Ver dados iniciais de todas as entidades

In [2]:
for ep in ['drivers','teams','seasons','circuits','races','contracts','results']:
    show(f'GET /{ep}', requests.get(f'{BASE}/{ep}'))

### GET /drivers | status 200
[{'date_of_birth': '1997-09-30',
  'full_name': 'Max Verstappen',
  'id': 1,
  'nationality': 'NLD'},
 {'date_of_birth': '1997-10-16',
  'full_name': 'Charles Leclerc',
  'id': 2,
  'nationality': 'MCO'},
 {'date_of_birth': None,
  'full_name': 'Oscar Piastri',
  'id': 3,
  'nationality': 'AUS'}]
### GET /teams | status 200
[{'base_country': 'AUT',
  'founded_year': 2005,
  'id': 1,
  'logo_url': 'https://upload.wikimedia.org/wikipedia/en/f/f1/Red_Bull_Racing.svg',
  'name': 'Red Bull Racing',
  'principal': 'Christian Horner'},
 {'base_country': 'ITA',
  'founded_year': 1950,
  'id': 2,
  'logo_url': 'https://upload.wikimedia.org/wikipedia/en/9/93/Scuderia_Ferrari_Logo.svg',
  'name': 'Scuderia Ferrari',
  'principal': 'Frédéric Vasseur'},
 {'base_country': 'GBR',
  'founded_year': None,
  'id': 3,
  'logo_url': None,
  'name': 'McLaren',
  'principal': None}]
### GET /seasons | status 200
[{'description': '74ª temporada do Campeonato Mundial de Fórmula 1

## 2. Driver CRUD completo

In [3]:
# CREATE
driver={'full_name':'Lewis Hamilton','nationality':'GBR','date_of_birth':'1985-01-07'}
r=requests.post(f'{BASE}/drivers',data=driver); show('POST /drivers',r)
driver_id=r.json()['id']
# GET
show('GET /drivers/{id}', requests.get(f'{BASE}/drivers/{driver_id}'))
# UPDATE
upd={'full_name':'Lewis Carl Hamilton'}
show('PUT /drivers', requests.put(f'{BASE}/drivers/{driver_id}',data=upd))

### POST /drivers | status 201
{'date_of_birth': '1985-01-07',
 'full_name': 'Lewis Hamilton',
 'id': 4,
 'nationality': 'GBR'}
### GET /drivers/{id} | status 200
{'date_of_birth': '1985-01-07',
 'full_name': 'Lewis Hamilton',
 'id': 4,
 'nationality': 'GBR'}
### PUT /drivers | status 200
{'date_of_birth': '1985-01-07',
 'full_name': 'Lewis Carl Hamilton',
 'id': 4,
 'nationality': 'GBR'}


In [4]:
# DELETE
show('DELETE /drivers', requests.delete(f'{BASE}/drivers/{driver_id}'))
# Create again for later use
driver_resp=requests.post(f'{BASE}/drivers',data=driver); driver_id=driver_resp.json()['id']

### DELETE /drivers | status 204



## 3. Team CRUD completo

In [ ]:
team={'name':'Mercedes','base_country':'DEU','principal':'Toto Wolff','founded_year':1954}
r=requests.post(f'{BASE}/teams',data=team); show('POST /teams',r)
team_id=r.json()['id']
# Duplicate name (should fail)
show('POST duplicate team', requests.post(f'{BASE}/teams',data=team))

show('PUT /teams', requests.put(f'{BASE}/teams/{team_id}',data={'principal':'James Allison'}))

### POST /teams | status 201
{'base_country': 'DEU',
 'founded_year': 1954,
 'id': 4,
 'logo_url': None,
 'name': 'Mercedes',
 'principal': 'Toto Wolff'}
### POST duplicate team | status 400
{'message': '(psycopg2.errors.UniqueViolation) ERRO:  duplicar valor da chave '
            'viola a restrição de unicidade "uq_team_name"\n'
            'DETAIL:  Chave (name)=(Mercedes) já existe.\n'
            '\n'
            '[SQL: INSERT INTO teams (name, logo_url, base_country, principal, '
            'founded_year) VALUES (%(name)s, %(logo_url)s, %(base_country)s, '
            '%(principal)s, %(founded_year)s) RETURNING teams.id]\n'
            "[parameters: {'name': 'Mercedes', 'logo_url': None, "
            "'base_country': 'DEU', 'principal': 'Toto Wolff', 'founded_year': "
            '1954}]\n'
            '(Background on this error at: https://sqlalche.me/e/20/gkpj)'}
### PUT /teams | status 422
<!doctype html>
<html lang=en>
<title>422 Unprocessable Entity</title>
<h1>Unprocessab

## 4. Season CRUD

In [6]:
season={'year':2028,'start_date':'2028-03-05','description':'78ª temporada'}
r=requests.post(f'{BASE}/seasons',data=season); show('POST /seasons',r)
season_id=r.json()['id']

### POST /seasons | status 201
{'description': '78ª temporada',
 'id': 3,
 'start_date': '2028-03-05',
 'year': 2028}


## 5. Circuit CRUD

In [ ]:
circuit={'name':'Yas Marina','country':'ARE','length_km':5.281,'map_url':'https://...'}
r=requests.post(f'{BASE}/circuits',data=circuit); show('POST /circuits',r)
circuit_id=r.json()['id']

show('PUT /circuits', requests.put(f'{BASE}/circuits/{circuit_id}',data={'length_km':5.305}))

### POST /circuits | status 201
{'country': 'ARE',
 'id': 3,
 'image_url': None,
 'length_km': 5.281,
 'map_url': 'https://...',
 'name': 'Yas Marina'}
### PUT /circuits | status 422
<!doctype html>
<html lang=en>
<title>422 Unprocessable Entity</title>
<h1>Unprocessable Entity</h1>
<p>The request was well-formed but was unable to be followed due to semantic errors.</p>



## 6. Race CRUD e validação de data

In [8]:
race={'name':'Abu Dhabi GP','race_date':'2028-12-10','season_id':season_id,'circuit_id':circuit_id,'laps':58,'weather':'Dry'}
r=requests.post(f'{BASE}/races',data=race); show('POST /races',r)
race_id=r.json()['id']
# Tentar data antes da temporada (erro)
bad_race=dict(race); bad_race['race_date']='2027-01-01'
show('POST invalid race date', requests.post(f'{BASE}/races',data=bad_race))

### POST /races | status 201
{'circuit_id': 3,
 'id': 3,
 'laps': 58,
 'name': 'Abu Dhabi GP',
 'race_date': '2028-12-10',
 'season_id': 3,
 'weather': 'Dry'}
### POST invalid race date | status 400
{'message': 'race_date cannot be before season start date'}


## 7. DriverContract CRUD e validações

In [9]:
contract={'season_id':season_id,'team_id':team_id,'driver_id':driver_id,'number':44,'salary_musd':55}
r=requests.post(f'{BASE}/contracts',data=contract); show('POST /contracts',r)
contract_id=r.json()['id']
# Tentar número duplicado
dup_contract=dict(contract); dup_contract['driver_id']=driver_id; dup_contract['number']=44
show('POST duplicate number', requests.post(f'{BASE}/contracts',data=dup_contract))

### POST /contracts | status 201
{'driver_id': 5,
 'id': 4,
 'number': 44,
 'salary_musd': 55.0,
 'season_id': 3,
 'team_id': 4}
### POST duplicate number | status 400
{'message': 'driver already contracted in this season'}


## 8. Result CRUD e validações de pontos

In [10]:
result={'race_id':race_id,'team_id':team_id,'driver_id':driver_id,'position':1,'points':25,'fastest_lap':True}
r=requests.post(f'{BASE}/results',data=result); show('POST /results',r)
result_id=r.json()['id']
# Pontos fora da faixa
bad_result=dict(result); bad_result['points']=30
show('POST pontos>25', requests.post(f'{BASE}/results',data=bad_result))

### POST /results | status 201
{'driver_id': 5,
 'fastest_lap': True,
 'id': 4,
 'points': 25,
 'position': 1,
 'race_id': 3,
 'team_id': 4}
### POST pontos>25 | status 422
<!doctype html>
<html lang=en>
<title>422 Unprocessable Entity</title>
<h1>Unprocessable Entity</h1>
<p>The request was well-formed but was unable to be followed due to semantic errors.</p>



## 9. Standings por temporada

In [11]:
show('Driver standings', requests.get(f'{BASE}/seasons/{season_id}/standings/drivers'))
show('Team standings', requests.get(f'{BASE}/seasons/{season_id}/standings/teams'))

### Driver standings | status 200
[{'driver_id': 5, 'full_name': 'Lewis Hamilton', 'points': 25}]
### Team standings | status 200
[{'name': 'Mercedes', 'points': 25, 'team_id': 4}]


## 10. Tentativa de apagar piloto com resultados (erro esperado)

In [12]:
show('DELETE driver with results', requests.delete(f'{BASE}/drivers/{driver_id}'))

### DELETE driver with results | status 500
<!doctype html>
<html lang=en>
  <head>
    <title>sqlalchemy.exc.IntegrityError: (psycopg2.errors.NotNullViolation) ERRO:  o valor nulo na coluna &#34;driver_id&#34; da relação &#34;results&#34; viola a restrição de não-nulo
DETAIL:  Registro que falhou contém (4, 3, 4, null, 1, 25, t).

[SQL: UPDATE results SET driver_id=%(driver_id)s WHERE results.id = %(results_id)s]
[parameters: {&#39;driver_id&#39;: None, &#39;results_id&#39;: 4}]
(Background on this error at: https://sqlalche.me/e/20/gkpj)
 // Werkzeug Debugger</title>
    <link rel="stylesheet" href="?__debugger__=yes&amp;cmd=resource&amp;f=style.css">
    <link rel="shortcut icon"
        href="?__debugger__=yes&amp;cmd=resource&amp;f=console.png">
    <script src="?__debugger__=yes&amp;cmd=resource&amp;f=debugger.js"></script>
    <script>
      var CONSOLE_MODE = false,
          EVALEX = true,
          EVALEX_TRUSTED = false,
          SECRET = "DMzP1w0YVZZ7i2qAPXzb";
    </scrip